In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [20]:
who_regions = pd.read_csv('data/who-regions.csv')
who_regions.head()

,Entity,Code,Year,WHO region
0,Afghanistan,AFG,2017,Eastern Mediterranean
1,Albania,ALB,2017,Europe
2,Algeria,DZA,2017,Africa
3,Andorra,AND,2017,Europe
4,Angola,AGO,2017,Africa


In [70]:
df = pd.read_csv('data/included_docs.csv').rename(columns={"prediction": "0 - relevance - prediction"})
df.head()

,id,content,title,PY,wosarticle__de,wosarticle__wc,relevant,seen,1 - Adaptation,1 - Impacts,1 - Mitigation,wosarticle__dt,tslug,UT__UT,0 - relevance - prediction,1 - Mitigation - prediction,1 - Adaptation - prediction,1 - Impacts - prediction
0,4963912,Objective To explore the association between n...,Exploration of the association between meteoro...,2021.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,explorationoftheassociationbetweenmeteorologic...,2-s2.0-85123003389,0.790797,0.010609,0.006835,0.980508
1,4729030,Background. Acute gastroenteritis is a cause o...,ACUTE GASTROENTERITIS IN 3 COMMUNITY-BASED NUR...,1995.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,acutegastroenteritisin3communitybasednursinghomes,WOS:A1995RY60500014,0.714243,0.024695,0.073035,0.910536
2,4762134,Background: Despite declining age-adjusted str...,Mortality and Predictors of Death 1 Month and ...,2010.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,mortalityandpredictorsofdeath1monthand3yearsaf...,WOS:000273552100004,0.554120,0.029360,0.061142,0.938161
3,678134,Weather factors are widely studied for their e...,Statistical Modeling Reveals the Effect of Abs...,2014.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Article,statisticalmodelingrevealstheeffectofabsoluteh...,WOS:000337735100017,0.957414,0.004876,0.007306,0.992824
4,304387,The aim of this study was to determine the hea...,A systematic review of the health effects of p...,2015.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,Review,asystematicreviewofthehealtheffectsofprenatale...,WOS:000364521200008,0.588936,0.003917,0.044029,0.944127


In [71]:
places = pd.read_csv('data/clean_place_df.csv')
if "WHO region" not in places.columns:
    places = places.merge(who_regions.rename(columns={"Code": "country_code3"})[["country_code3","WHO region"]])
places.head()

,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id,continent,WHO region
0,Shanghai,"[{'start': 162, 'end': 170}]",CHN,0.963741,Shanghai,31.22222,121.45806,CHN,1796236.0,Shanghai,P,PPLA,4963912,Asia,Western Pacific
1,China,"[{'start': 113, 'end': 118}]",CHN,0.637832,Hubei,30.52047,114.39637,CHN,6620465.0,China University of Geosciences,S,SCHC,3388074,Asia,Western Pacific
2,China,"[{'start': 919, 'end': 924}]",CHN,0.655191,Hubei,30.52047,114.39637,CHN,6620465.0,China University of Geosciences,S,SCHC,4351843,Asia,Western Pacific
3,Licheng,"[{'start': 118, 'end': 125}]",CHN,0.930774,Guangdong,23.29549,113.82465,CHN,1803791.0,Licheng,P,PPLA3,1046237,Asia,Western Pacific
4,China,"[{'start': 127, 'end': 132}]",CHN,0.665504,Hubei,30.52047,114.39637,CHN,6620465.0,China University of Geosciences,S,SCHC,1046237,Asia,Western Pacific


In [72]:
dais = pd.read_csv('data/institutional_locations.csv')
dais = (dais
        .rename(columns={"alpha_3": "country_code3"})
        .merge(who_regions.rename(columns={"Code": "country_code3"})[["country_code3","WHO region"]])
       )

auth_strings = dais.drop_duplicates(['AU','doc_id']).sort_values('position').groupby('doc_id')['AU'].agg(
    lambda x: "; ".join(x)
).reset_index().rename(columns={"doc_id": "id", "AU": "authors"})

dais.head()

,id,doc_id,surname,initials,AU,AF,institution,position,country,country_code3,name,alpha_2,WHO region
0,1217824,562,NaN,NaN,"Pinkerton, KE","Pinkerton, Kent E.","Univ Calif Davis, Ctr Hlth & Environm, Davis, ...",4,USA,USA,United States,US,Americas
1,1217742,562,NaN,NaN,"Thurston, GD","Thurston, George D.","NYU, Dept Environm Med, Sch Med, New York, NY ...",2,USA,USA,United States,US,Americas
2,1217830,562,NaN,NaN,"Pinkerton, KE","Pinkerton, Kent E.","Univ Calif Davis, John Muir Inst Environm, Dav...",4,USA,USA,United States,US,Americas
3,1217730,562,NaN,NaN,"Rice, MB","Rice, Mary B.","Harvard Univ, Beth Israel Deaconess Med Ctr, S...",1,USA,USA,United States,US,Americas
4,1217695,562,NaN,NaN,"Rice, MB","Rice, Mary B.","Massachusetts Gen Hosp, Pulm & Crit Care Unit,...",1,USA,USA,United States,US,Americas


In [73]:
auth_strings

,id,authors
0,562,"Rice, MB; Thurston, GD; Balmes, JR; Pinkerton, KE"
1,1061,"Zhu, YH; Yang, LX; Meng, CP; Yuan, Q; Yan, C; ..."
2,1285,"Chung, M; Wang, DD; Rizzo, AM; Gachette, D; De..."
3,1672,"Green, D; Bambrick, H; Tait, P; Goldie, J; Sch..."
4,1682,"Kais, SM; Islam, MS"
...,...,...
25764,5086783,"Ashton, RA; Bennett, A; Al-Mafazy, AW; Abass, ..."
25765,5086790,"Malik, EM; Abdalla, AM; Mohamednour, SS; Osman..."
25766,5086811,"Mukhopadhyay, C; Shaw, T; Varghese, GM; Dance,..."
25767,5086814,"Makwana, N"


In [74]:
if "authors" not in df.columns:
    df = df.merge(auth_strings)

pred_cols = [x for x in df.columns if "pred" in x]
df = df[['id','content','title','PY','authors'] + pred_cols]
df.to_csv('data/included_studies.csv',index=False)
df.head()

,id,content,title,PY,authors,0 - relevance - prediction,1 - Mitigation - prediction,1 - Adaptation - prediction,1 - Impacts - prediction
0,4762134,Background: Despite declining age-adjusted str...,Mortality and Predictors of Death 1 Month and ...,2010.0,"Koton, S; Tanne, D; Green, MS; Bornstein, NM",0.554120,0.029360,0.061142,0.938161
1,678134,Weather factors are widely studied for their e...,Statistical Modeling Reveals the Effect of Abs...,2014.0,"Xu, HY; Fu, XJ; Lee, LKH; Ma, S; Goh, KT; Wong...",0.957414,0.004876,0.007306,0.992824
2,1622376,Background Malaria is an important cause of il...,Mosquito larval source management for controll...,2013.0,"Tusting, LS; Thwing, J; Sinclair, D; Fillinger...",0.604748,0.060423,0.312601,0.700792
3,3388074,Background Aedes albopictus is naturally infec...,Identification and molecular characterization ...,2020.0,"Hu, YP; Xi, ZY; Liu, XB; Wang, J; Guo, YH; Ren...",0.563156,0.013087,0.017681,0.963909
4,670073,Determining the role of weather in waterborne ...,Analytical studies assessing the association b...,2015.0,"Herrador, BRG; de Blasio, BF; MacDonald, E; Ni...",0.865812,0.005846,0.031864,0.974864


In [75]:
with pd.ExcelWriter('data/institutions_authors.xlsx') as writer: 

    # For each who region
    for i, who in enumerate(who_regions['WHO region'].unique()):
        print(who)
        # get the papers mentioning a place in this region
        dids = set(places[places['WHO region']==who].doc_id)

        top_institutions = (dais[dais.doc_id.isin(dids)]
                            .groupby('institution')['doc_id']
                            .nunique()
                            .sort_values(ascending=False)
                            .to_frame("n_studies")
                            .reset_index()
                           )
        top_institutions.to_excel(writer, sheet_name=f"Institutions about {who}", index=False)
        
        top_authors = (dais[dais.doc_id.isin(dids)]
                            .groupby('AU')['doc_id']
                            .nunique()
                            .sort_values(ascending=False)
                            .to_frame("n_studies")
                            .reset_index()
                           )
        top_authors.to_excel(writer, sheet_name=f"Authors about {who}", index=False)
        
        # Get the papers with an affiliation from this region
        dids = set(dais[dais['WHO region']==who].doc_id)

        top_institutions = (dais[dais.doc_id.isin(dids)]
                            .groupby('institution')['doc_id']
                            .nunique()
                            .sort_values(ascending=False)
                            .to_frame("n_studies")
                            .reset_index()
                           )
        top_institutions.to_excel(writer, sheet_name=f"Institutions from {who}", index=False)
        
        top_authors = (dais[dais.doc_id.isin(dids)]
                            .groupby('AU')['doc_id']
                            .nunique()
                            .sort_values(ascending=False)
                            .to_frame("n_studies")
                            .reset_index()
                           )
        top_authors.to_excel(writer, sheet_name=f"Authors from {who}", index=False)        
        

Eastern Mediterranean


/home/max/software/py39/lib/python3.9/site-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


Europe
Africa
Americas
Western Pacific
South-East Asia
